# SSC-OMP on synthetic data

In this example:
- We consider $\mathbb{R}^M$ as our ambient space of data with $M=9$.
- We sample $K=5$ low dimensional subspaces from it. Each subspace is represented by an orthonormal basis for it.
- Each subspace is of dimension $D=6$. 
- We then sample $S_k = 10000$ points from each subspace and put them together in a data matrix $X$.
- Total number of points is $S=50000$.
- We assign a (true) label to each data point point.
- We then use OMP algorithm to build a self expressive representation of each point in terms of other points. 
- If things go well, we hope that OMP will choose points from the same subspace to create a sparse representation of each point in $X$. 
- Since each subspace is $D$ dimensional, each point will be represented in terms of $D$ other points. 
- We assemble these representations into a sparse matrix $Z$ which has exactly $DS$ non-zero entries. This and following steps use the `jax.experimental.sparse.BCOO` for efficiently storing the sparse matrices. 
- We construct an affinity matrix $W = |C| + |C^T|$. 
- We run spectral clustering to cluster these data points into $K$ clusters using the affinity matrix $W$ as input.
- We use Hungarian algorithm to map the true labels with the labels assigned by spectral clustering. 
- We then measure the clustering error (number of points not assigned into the correct cluster). 
- We also preserve the subspace preservation ratio. i.e. in the $D$-sparse representation for each point, how many of the contributing points were from the same subspace.




In [1]:
from jax.config import config
config.update("jax_enable_x64", True)
import jax.numpy as jnp
import cr.sparse as crs
import cr.sparse.data as crdata
import cr.sparse.la.subspaces
import cr.sparse.cluster as cluster
import cr.sparse.cluster.ssc as ssc
import cr.sparse.cluster.spectral as spectral

In [2]:
# Ambient space dimension
M = 9
# Number of subspaces
K = 5
# common dimension for each subspace
D = 6
# Number of points per subspace
Sk = 10000

In [3]:
# select orthnormal bases for K random subspaces of dimension D  
bases = crdata.random_subspaces_jit(crs.KEYS[0], M, D, K)
cluster_sizes = jnp.ones(K, dtype=int) * Sk
# Select Sk points for each subspace
X = crdata.uniform_points_on_subspaces_jit(crs.KEYS[1], bases, Sk)
# cluster labels for each of them
true_labels = jnp.repeat(jnp.arange(K), Sk)

In [4]:
batch_size = 800
# build sparse representations in batches
Z, I, R = ssc.batch_build_representation_omp_jit(X, D, batch_size)

In [5]:
%timeit ssc.batch_build_representation_omp_jit(X, D, batch_size)[0].block_until_ready()

10.2 s ± 50 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
# Combine values and indices to form full representation
Z_sp = ssc.sparse_to_bcoo(Z, I)

In [7]:
# Build the affinity matrix
affinity = ssc.rep_to_affinity(Z_sp)

In [8]:
# Perform the spectral clustering on the affinity matrix
res = spectral.normalized_symmetric_sparse_fast_k_jit(crs.KEYS[2], affinity, K)

In [9]:
%timeit spectral.normalized_symmetric_sparse_fast_k_jit(crs.KEYS[2], affinity, K).assignment.block_until_ready()

867 ms ± 1.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
# Predicted cluster labels
pred_labels = res.assignment

In [11]:
clust_err = cluster.clustering_error_k(true_labels, pred_labels, K)
print(clust_err)

num_missed: 857, error: 0.02, error_perc: 1.71
[3 2 4 1 0]
0=>3, 1=>2, 2=>4, 3=>1, 4=>0


In [12]:
# subspace preservation stats
spr_stats = ssc.sparse_subspace_preservation_stats_jit(Z, I, true_labels)
print(spr_stats)

spr_error: 0.02348690787086873, spr_flag: False, spr_perc: 92.134
